In [1]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision.io import read_image
from torchvision import models, transforms
import time

In [2]:
cd ..

/home/avani.gupta/cva5


In [3]:
f = open("data/dataset_info.txt", "r")
labels = f.readlines()[1:]
f.close()

name_label = {}
lb_list = []

for i in range(len(labels)):
    lb = labels[i].replace("\n", "").strip().split(". ")[1]
    lb_list.append(lb)
    name_label[lb] = 0
    
for i in range(len(lb_list)):
    name_label[lb_list[i]] += i
    
label_name = list(name_label.keys())


In [4]:
class LoadImageDataset(Dataset):
    def __init__(self, imgname_labels_file, img_dir, transform_data = None, train = True):
        self.df_imgname_labels = pd.read_csv(imgname_labels_file)[:100]
        self.img_dir = img_dir
        self.transform_data =  transform_data
        self.train = train
        
    def __len__(self):
        return len(self.df_imgname_labels)
    
    def __getitem__(self, idx):
        path = self.img_dir + self.df_imgname_labels.iloc[idx, 0]
        img =  read_image(path)/ 255.
        result = {"image": img, "label": -1}
        if self.train:
            lb = self.df_imgname_labels.iloc[idx, 1]
            result["label"] = lb
            if self.transform_data:
                result["label"] = self.transform_data['labels'](result["label"])
                result["image"] = self.transform_data['images'](result["image"])
        else:
            lb = 0
            result["label"] = lb
            if self.transform_data: 
                result["image"] = self.transform_data['images'](result["image"])
        
        return result

In [5]:
class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.resent152_model = models.resnet152(pretrained = True)
        self.resent152_model.fc = nn.Linear(in_features = 2048, out_features= len(name_label), bias = True)
        
#         for p in self.resent152_model.parameters():
#             p.requires_grad = False
        
#         self.end_layers = nn.Sequential(
#             nn.Linear(in_features = 1000, out_features = 700, bias = True),
#             nn.ReLU(inplace=True),
#             nn.Dropout(0.2),
#             nn.Linear(in_features = 700, out_features = 300, bias = True),
#             nn.ReLU(inplace=True),
#             nn.Dropout(0.2),
#             nn.Linear(in_features = 300, out_features = 100, bias = True),
#             nn.ReLU(inplace=True),
#             nn.Dropout(0.1),
#             nn.Linear(in_features = 100, out_features = len(name_label), bias = True)
#         )
        
    def forward(self, x):
        return self.resent152_model(x)

In [6]:
def train(path = "data/", batch_size = 8, epochs = 20):
    
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print("Using {} device".format(device))
    
    model = MyNetwork().to(device)
    print(model)
    
    transform_data =  {
        'images': 
                transforms.Compose([
                transforms.CenterCrop((224, 224)),
                transforms.Resize((256, 256)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                transforms.RandomApply(
                    torch.nn.ModuleList([transforms.RandomHorizontalFlip(),
                    transforms.ColorJitter(),
                    transforms.RandomAffine(30, shear=0.2, scale = (0.8, 1.4)),
                    transforms.RandomGrayscale(p=0.9),
                    transforms.RandomPerspective()]), p=0.3),
                ]),
        'labels': 
                transforms.Compose([
                transforms.Lambda(lambda y: name_label[y])
                ])
    }
    
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    loss_fun = nn.CrossEntropyLoss()
    dataloader = DataLoader(LoadImageDataset(path + "train.csv", path + "train_images/", transform_data), batch_size = batch_size,  num_workers=2, shuffle=True)
    
    for epoch in range(epochs):
        total_loss = 0
        
        for  dic in dataloader:
            X = dic["image"].to(device)
            y = dic["label"].to(device)
            
            # forward pass
            y_preds = model(X)
            loss = loss_fun(y_preds, y)
            
            # backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
           
        total_loss /= len(dataloader.dataset)
        print("Epoch: "+ str(epoch+1) + " | loss: "+str(total_loss), flush = True)
    
    torch.save(model.state_dict(), "./resnet152_test.pth")
    print("Training completed")



In [7]:
epochs = 5
train(epochs = epochs)



Using cuda device
MyNetwork(
  (resent152_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsam

Epoch: 1 | loss: 0.5966356444358826
Epoch: 2 | loss: 0.4882220196723938
Epoch: 3 | loss: 0.49093305110931396
Epoch: 4 | loss: 0.4551620101928711
Epoch: 5 | loss: 0.452338855266571
Training completed


In [9]:
def test(batch_size = 8, path = "data/"):
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print("Using {} device".format(device))

    results = []
    model = MyNetwork()
    model.load_state_dict(torch.load("./resnet152_test.pth"))
    model = model.to(device)
    model.eval()
    
    transform_data = {
        'images':
            transforms.Compose([
            transforms.CenterCrop((224, 224)),
            transforms.Resize((256, 256)),
            transforms.Normalize(std=[0.229, 0.224, 0.225], mean=[0.485, 0.456, 0.406])
    ])}
    
    dataloader = DataLoader(LoadImageDataset(path + "test.csv", path + "test_images/", transform_data, train = False), batch_size = batch_size)
    
    
    with torch.no_grad():
        for dic in dataloader: 
            preds = model(dic["image"].to(device)).argmax(dim = 1)
            results.extend(preds.tolist())
    
    list_results = []
    for res in results:
        list_results.append(label_name[res])
        
    df = pd.DataFrame(list_results, columns=['ClassName'])
    df.to_csv("submission152_1.csv", index=False)
    
    
test()

Using cuda device
